In [1]:
import sys
import os
sys.path.append('../')
os.chdir('../')

import pandas as pd
import numpy as np
import shutil
from pathlib import Path
from datetime import datetime
import time
import threading
import json
import requests
import threading
import re
# from src.cima import cima_api as cimaAPI
from src import utils  

cima_url = 'https://cima.aemps.es/cima/rest/'
timeout_value = 10


GET medicamentos?{condiciones} da medicamentos con nombres
GET medicamento?{condiciones } da medicamento con nombre.
GET docSegmentado/secciones/:tipoDoc?{condiciones } secciones sin info
GET docSegmentado/contenido/:tipoDoc?{condiciones } secciones con info
POST buscarEnFichaTecnica

GET https://cima.aemps.es/cima/rest/docSegmentado/contenido/1?nregistro=64014&seccion=   
GET https://cima.aemps.es/cima/rest/docSegmentado/secciones/1?nregistro=64014&seccion=   

https://cima.aemps.es/cima/rest/buscarEnFichaTecnica
[{
"seccion":"4.1",
"texto":"cáncer",
"contiene":1
}]


### API

In [ ]:
class MyCimaAPI:
    # TODO:

    """API para Cima.

            Atributos
            ----------

            Constructor
            ------------
                MyCimaAPI()

            Métodos
            -------
                create_url: 
                get_medicamentos: 
                get_medicamento: 
                get_info_medicamento: 
                get_info_secciones: 
                buscarEnFichaTecnica:
    """

    def __init__(self):
        pass

    def create_url(self,kind,list_var={}):

        """Dado el tipo de url, crea el link para realizar la peticion.

            Parametros
            ----------
                kind: String
                    Tipo de petición a realizar en la API. Ex: Eutirox

            Retorna
            -------
                url: String
                    Url para realizar la peticion, Ex: https://cima.aemps.es/cima/rest/medicamentos?&nombre=EUTIROX

        """

        url = cima_url + kind

        for index, (key, value) in enumerate(list_var.items()):
            if isinstance(value,int):
                url += "&" + key + "=" + str(value)
            elif isinstance(value,str):
                url += "&" + key + "=" + value
        print(url)
        return url

    def get_medicamentos(self,name=None,page=1):
        """Devuelve informacion del medicamento proporcionado, puede ser mas de un tipo, dependiendo de los gramos, mililitros etc.

            Parametros
            ----------
                name: String
                    Nombre del medicamento. Ex: Eutirox
                page: String
                    Nombre del medicamento. Ex: Eutirox

            Retorna
            -------
                text: json
                    Contiene el medicamento encontrado que coincide con el nombre proporcionado.

                resp_code: int
                    Codigo de respuesta de nuestra consulta. Ex: 200
        """

        dic_var = {
            "nombre": name 
        }

        url = self.create_url("medicamentos?",dic_var)

        params = {
            'pagina': page,      # Setting the current page number
            'tamanioPagina': 25  # Setting the number of items per page
        }

        # Send query, 
        response = requests.get(url, params=params,timeout=timeout_value)
        resp_code = response.status_code
        resp_message = response.text

        # If all OK, get info
        products = None
        if resp_code == 200:
            text = json.loads(resp_message)
        else:
            return None
        
        return text, resp_code

    def get_medicamentos_all(self,name=None):
        """Devuelve informacion de medicamentos que contienen el nombre proporcionado, puede ser mas de un tipo, dependiendo de los gramos, mililitros etc.

            Parametros
            ----------
                nombre: String
                    Nombre del medicamento. Ex: Eutirox

            Retorna
            -------
                text: list
                    Contiene la lista de medicamentos encontrado que coincide con el nombre proporcionado.

        """

        dic_var = {
            "nombre": name 
        }

        url = self.create_url("medicamentos?",dic_var)

        # Send query, 
        response = requests.get(url, timeout=timeout_value)
        resp_message = response.json()

        total_pages = round(resp_message["totalFilas"] / resp_message["tamanioPagina"])

        list_objects = []

        # Looping through each page
        for page in range(1, total_pages + 1):
            # Parameters for the request
            params = {
                'pagina': page,      # Setting the current page number
                'tamanioPagina': 25  # Setting the number of items per page
            }

            # Making the request
            response = requests.get(url, params=params)

            # Check if the request was successful
            if response.status_code == 200:
                data = response.json()
                if not data["resultados"]:
                    break
                else:
                    list_objects += data["resultados"]
            else:
                print(f"Failed to retrieve data for page {page}")

        return list_objects

    def get_medicamento(self,cn=None,nregister=None):
        """Devuelve informacion del medicamento proporcionado, que coincide con el numero de registro, tambien cuenta con los principios activos.

            Parametros
            ----------
                cn: int
                    Codigo nacional. Ex: ?
                nregister: int
                    Numero de registro. Ex: 64014

            Retorna
            -------
                text: json
                    Contiene el medicamento encontrado que coincide con el numero de registro.

                resp_code: int
                    Codigo de respuesta de nuestra consulta. Ex: 200
        """

        dic_var = {
            "cn": cn,
            "nregistro": nregister
        }

        url = self.create_url("medicamento?",dic_var)

        response = requests.get(url, timeout=timeout_value)
        resp_code = response.status_code

        # If all OK, get info
        if resp_code == 200:
            text = response.json()
        else:
            return None
        
        return text, resp_code

    def get_info_medicamento_section(self,typeDoc=1,nregister=None,section=""):
        """Devuelve una lista de secciones con su informacion para el tipo de documento asociado al medicamento por número de registro.

            Parametros
            ----------
                typeDoc: int
                    Tipo de documento. Ex: 1/2
                        1: Ficha técnica
                        2: Prospecto
                nregister: int
                    Numero de registro. Ex: 64014
                section: String
                    Seccion a solicitar. Ex: 4.8 or None

            Retorna
            -------
                text: json
                    Contiene el medicamento encontrado que coincide con el numero de registro.

                resp_code: int
                    Codigo de respuesta de nuestra consulta. Ex: 200
        """

        dic_var = {
            "nregistro": nregister,
            "seccion": section
        }
        headers = {
            'Accept':"application/json"
        }

        url = self.create_url("docSegmentado/contenido/" + str(typeDoc) + "?" ,dic_var)

        # Send query, 
        response = requests.get(url, timeout=timeout_value, headers=headers)
        resp_code = response.status_code

        # If all OK, get info
        if resp_code == 200:
            text = response.json()
        else:
            return None

        return text,resp_code

    def get_info_secciones(self,typeDoc=None,nregister=None):
        """Devuelve una lista con las secciones existentes para un tipo de documento y un medicamento.

            Parametros
            ----------
                tipoDoc: int
                    Tipo de documento. Ex: 1/2
                        1: Ficha técnica
                        2: Prospecto
                nregistro: int
                    Numero de registro. Ex: 64014

            Retorna
            -------
                text: json
                    Contiene el medicamento encontrado que coincide con el numero de registro.

                resp_code: int
                    Codigo de respuesta de nuestra consulta. Ex: 200
        """

        dic_var = {
            "nregistro": nregister,
        }

        url = self.reate_url("docSegmentado/secciones/" + str(typeDoc) + "?" ,dic_var)

        # Send query, 
        response = requests.get(url, timeout=timeout_value)
        resp_code = response.status_code

        # If all OK, get info
        if resp_code == 200:
            text = response.json()
        else:
            return None

        return text,resp_code

    def buscarEnFichaTecnica(self,body):
        """Recibe en formato JSON como cuerpo de la petición una serie de texto a buscar en las secciones y devuelve una lista de medicamentos.

            Parametros
            ----------
                body: List
                    Parametros para realizar la busqueda. Ex: # body = [{"seccion":"4.1","texto":"cáncer","contiene":1}]

            Retorna
            -------
                text: list
                    Contiene la lista de medicamentos encontrado que coincide con el body proporcionado.

        """

        url = self.create_url("buscarEnFichaTecnica")
        headers = {'Content-Type': 'application/json'}

        # Realizar la solicitud POST
        response = requests.post(url, data=json.dumps(body), headers=headers, timeout=timeout_value)
        resp_message = response.json()

        print(resp_message["totalFilas"])
        total_pages = round(resp_message["totalFilas"] / resp_message["tamanioPagina"])

        list_objects = []

        # Looping through each page
        for page in range(1, total_pages + 1):
            # Parameters for the request
            params = {
                'pagina': page,      # Setting the current page number
                'tamanioPagina': 25  # Setting the number of items per page
            }

            # Making the request
            response = requests.post(url, data=json.dumps(body), headers=headers, params=params)

            # Check if the request was successful
            if response.status_code == 200:
                data = response.json()
                if not data["resultados"]:
                    break
                else:
                    list_objects += data["resultados"]
            else:
                print(f"Failed to retrieve data for page {page}")
                
        return list_objects

    def registroCambios(self,date,nregister=""):
        """Recibe en formato JSON como cuerpo de la petición una serie de texto a buscar en las secciones y devuelve una lista de medicamentos.

            Parametros
            ----------
                date: String
                   Fecha desde la que se quiere conoecer los cambios. Ex: 11/11/2023
                nregistro: int
                    Numero de registro. Ex: 64014

            Retorna
            -------
                text: list
                    Contiene la lista de medicamentos que han recibido algun cambio desde la fecha indicada.
        """

        dic_var = {
            "fecha":date,
            "nregistro": nregister
        }

        url = self.create_url("registroCambios?",dic_var)

        # Send query, 
        response = requests.post(url, timeout=timeout_value)
        resp_message = response.json()

        print(resp_message["totalFilas"])
        total_pages = round(resp_message["totalFilas"] / resp_message["tamanioPagina"])

        list_objects = []

        # Looping through each page
        for page in range(1, total_pages + 1):
            # Parameters for the request
            params = {
                'pagina': page,      # Setting the current page number
                'tamanioPagina': 25  # Setting the number of items per page
            }

            # Making the request
            response = requests.post(url, params=params)

            # Check if the request was successful
            if response.status_code == 200:
                data = response.json()
                if not data["resultados"]:
                    break
                else:
                    list_objects += data["resultados"]
            else:
                print(f"Failed to retrieve data for page {page}")
                
        return list_objects

In [3]:
from bs4 import BeautifulSoup
import requests
import os
import json

def extract_and_clean_text(html_content):
    # Parse the HTML content
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Extract text and filter out empty lines
    text = soup.get_text(separator='\n').strip()
    lines = [line.strip() for line in text.split('\n') if line.strip()]
    
    # Join the lines back into a single string
    cleaned_text = '\n'.join(lines)
    return cleaned_text

### Experimentos

In [ ]:
cima = MyCimaAPI()

In [8]:
# cima.get_medicamentos(name="EUTIROX")
# cima.get_medicamentos_all(name="EUTIROX")
cima.get_medicamento(cn="",nregister="64014")
# cima.get_info_medicamento(typeDoc=1,nregister="64014",section="")
# cima.get_info_secciones(typeDoc=1,nregister="64014")

# body = [{
# "seccion":"4.1",
# "texto":"cáncer",
# "contiene":1
# }]
# cima.buscarEnFichaTecnica(body)

https://cima.aemps.es/cima/rest/medicamento?&cn=64014&nregistro=64014


In [111]:
text, status = cima.get_medicamentos(name="EUTIROX")

print(text["resultados"][0]["nregistro"])
print(text["resultados"][0]["nombre"].split(" ")[0])
print(text["resultados"][0]["docs"][-1]["fecha"])
print(text["resultados"][0])

https://cima.aemps.es/cima/rest/medicamentos?&nombre=EUTIROX
64014
EUTIROX
1699917809000
{'nregistro': '64014', 'nombre': 'EUTIROX 100 microgramos COMPRIMIDOS', 'labtitular': 'Merck S.L.', 'cpresc': 'Medicamento Sujeto A Prescripción Médica. Tratamiento De Larga Duración', 'estado': {'aut': 993506400000}, 'comerc': True, 'receta': True, 'generico': False, 'conduc': False, 'triangulo': False, 'huerfano': False, 'biosimilar': False, 'nosustituible': {'id': 2, 'nombre': 'Medicamentos con principios activos de estrecho margen terapéutico'}, 'psum': False, 'notas': True, 'materialesInf': False, 'ema': False, 'docs': [{'tipo': 1, 'url': 'https://cima.aemps.es/cima/pdfs/ft/64014/FT_64014.pdf', 'urlHtml': 'https://cima.aemps.es/cima/dochtml/ft/64014/FT_64014.html', 'secc': True, 'fecha': 1686090551000}, {'tipo': 2, 'url': 'https://cima.aemps.es/cima/pdfs/p/64014/P_64014.pdf', 'urlHtml': 'https://cima.aemps.es/cima/dochtml/p/64014/P_64014.html', 'secc': True, 'fecha': 1699917809000}], 'fotos': 

In [112]:
list_elements = cima.get_medicamentos_all(name="EUTIROX")
print(list_elements)
print(len(list_elements))
for element in list_elements:
    print(element["nombre"].split(" ")[0])

https://cima.aemps.es/cima/rest/medicamentos?&nombre=EUTIROX
[{'nregistro': '64014', 'nombre': 'EUTIROX 100 microgramos COMPRIMIDOS', 'labtitular': 'Merck S.L.', 'cpresc': 'Medicamento Sujeto A Prescripción Médica. Tratamiento De Larga Duración', 'estado': {'aut': 993506400000}, 'comerc': True, 'receta': True, 'generico': False, 'conduc': False, 'triangulo': False, 'huerfano': False, 'biosimilar': False, 'nosustituible': {'id': 2, 'nombre': 'Medicamentos con principios activos de estrecho margen terapéutico'}, 'psum': False, 'notas': True, 'materialesInf': False, 'ema': False, 'docs': [{'tipo': 1, 'url': 'https://cima.aemps.es/cima/pdfs/ft/64014/FT_64014.pdf', 'urlHtml': 'https://cima.aemps.es/cima/dochtml/ft/64014/FT_64014.html', 'secc': True, 'fecha': 1686090551000}, {'tipo': 2, 'url': 'https://cima.aemps.es/cima/pdfs/p/64014/P_64014.pdf', 'urlHtml': 'https://cima.aemps.es/cima/dochtml/p/64014/P_64014.html', 'secc': True, 'fecha': 1699917809000}], 'fotos': [{'tipo': 'materialas', 'ur

In [113]:
text, status = cima.get_medicamento(cn="",nregister="64014")

https://cima.aemps.es/cima/rest/medicamento?&cn=&nregistro=64014


In [114]:
print(text)
print(text["pactivos"])
print(text["principiosActivos"])

{'nregistro': '64014', 'nombre': 'EUTIROX 100 microgramos COMPRIMIDOS', 'pactivos': 'LEVOTIROXINA SODICA', 'labtitular': 'Merck S.L.', 'cpresc': 'Medicamento Sujeto A Prescripción Médica. Tratamiento De Larga Duración', 'estado': {'aut': 993506400000}, 'comerc': True, 'receta': True, 'generico': False, 'conduc': False, 'triangulo': False, 'huerfano': False, 'biosimilar': False, 'nosustituible': {'id': 2, 'nombre': 'Medicamentos con principios activos de estrecho margen terapéutico'}, 'psum': False, 'notas': True, 'materialesInf': False, 'ema': False, 'docs': [{'tipo': 1, 'url': 'https://cima.aemps.es/cima/pdfs/ft/64014/FT_64014.pdf', 'urlHtml': 'https://cima.aemps.es/cima/dochtml/ft/64014/FT_64014.html', 'secc': True, 'fecha': 1686090551000}, {'tipo': 2, 'url': 'https://cima.aemps.es/cima/pdfs/p/64014/P_64014.pdf', 'urlHtml': 'https://cima.aemps.es/cima/dochtml/p/64014/P_64014.html', 'secc': True, 'fecha': 1699917809000}], 'fotos': [{'tipo': 'materialas', 'url': 'https://cima.aemps.es/

In [115]:
text, status = cima.get_info_medicamento_section(typeDoc=1,nregister="64014",section="")

https://cima.aemps.es/cima/rest/docSegmentado/contenido/1?&nregistro=64014&seccion=


In [116]:
print(text[17])
print(text[17]["contenido"])

{'seccion': '4.8', 'titulo': 'Reacciones adversas', 'contenido': '<div>\r\n    <p style="margin:0pt; text-align:justify; line-height:115%; font-size:11pt"><span style="font-family:Times New Roman; font-size:11pt">Cuando se excede el l&#237;mite de tolerancia individual de levotiroxina de sodio o cuando se produce sobredosis, especialmente si se aumenta la dosis demasiado r&#225;pidamente al inicio del tratamiento, es posible que aparezcan los siguientes s&#237;ntomas cl&#237;nicos t&#237;picos de hipertiroidismo: arritmias card&#237;acas (ej. fibrilaci&#243;n auricular y extras&#237;stoles), taquicardia, palpitaciones, angor, dolor de cabeza, debilidad muscular y calambres, rubor, fiebre, v&#243;mitos, alteraciones de la menstruaci&#243;n, pseudotumor cerebral, temblor, agitaci&#243;n, insomnio, hiperhidrosis, p&#233;rdida de peso y diarrea.</span></p>\r\n\r\n    <p style="margin:0pt; text-align:justify; line-height:115%; font-size:11pt"><span style="font-family:Times New Roman; font-s

In [117]:
extract_and_clean_text(text[17]["contenido"])

'Cuando se excede el límite de tolerancia individual de levotiroxina de sodio o cuando se produce sobredosis, especialmente si se aumenta la dosis demasiado rápidamente al inicio del tratamiento, es posible que aparezcan los siguientes síntomas clínicos típicos de hipertiroidismo: arritmias cardíacas (ej. fibrilación auricular y extrasístoles), taquicardia, palpitaciones, angor, dolor de cabeza, debilidad muscular y calambres, rubor, fiebre, vómitos, alteraciones de la menstruación, pseudotumor cerebral, temblor, agitación, insomnio, hiperhidrosis, pérdida de peso y diarrea.\nEn estos casos la dosis diaria debe disminuirse o suspenderse la medicación durante algunos días. Una vez hayan desaparecido las reacciones adversas, la terapia puede reanudarse con precaución.\nEn caso de hipersensibilidad a cualquiera de los componentes de Eutirox pueden aparecer especialmente reacciones alérgicas cutáneas (sarpullido, urticaria) o del tracto respiratorio. Se han descrito casos de angioedema.\nN

In [118]:
body = [{
"seccion":"4.1",
"texto":"cáncer",
"contiene":1
}]
list_objects = cima.buscarEnFichaTecnica(body)

https://cima.aemps.es/cima/rest/buscarEnFichaTecnica
75


In [119]:
print(len(list_objects))
print(list_objects)

75
[{'nregistro': '71857', 'nombre': 'ANASTROZOL SANDOZ 1 MG COMPRIMIDOS RECUBIERTOS CON PELÍCULA EFG', 'labtitular': 'Sandoz Farmaceutica S.A.', 'cpresc': 'Medicamento Sujeto A Prescripción Médica', 'estado': {'aut': 1302645600000}, 'comerc': True, 'receta': True, 'generico': True, 'conduc': False, 'triangulo': False, 'huerfano': False, 'biosimilar': False, 'nosustituible': {'id': 0, 'nombre': 'N/A'}, 'psum': False, 'notas': False, 'materialesInf': False, 'ema': False, 'docs': [{'tipo': 1, 'url': 'https://cima.aemps.es/cima/pdfs/ft/71857/FT_71857.pdf', 'urlHtml': 'https://cima.aemps.es/cima/dochtml/ft/71857/FT_71857.html', 'secc': True, 'fecha': 1624398171000}, {'tipo': 2, 'url': 'https://cima.aemps.es/cima/pdfs/p/71857/P_71857.pdf', 'urlHtml': 'https://cima.aemps.es/cima/dochtml/p/71857/P_71857.html', 'secc': True, 'fecha': 1624398183000}], 'fotos': [{'tipo': 'materialas', 'url': 'https://cima.aemps.es/cima/fotos/thumbnails/materialas/71857/71857_materialas.jpg', 'fecha': 15295048340

In [5]:
list_objects = cima.registroCambios("27/11/2023")

https://cima.aemps.es/cima/rest/registroCambios?&fecha=27/11/2023&nregistro=
315


In [7]:
len(list_objects)

315

### Read names

In [4]:
filename = "data/lista_nombre.txt"

with open(filename, 'r') as file:
    content = file.read()

# Split the content by ':' and remove any whitespace
names = [num.strip() for num in content.split(':')]

In [5]:
len(names)

12027

In [48]:
# Example usage
text = "COZAAR PLUS 50 mg/12,5 mg COMPRIMIDOS RECUBIERTOS CON PELICULA"
result = utils.extract_text_before_number(text)
print(result)  # Output: COZAAR PLUS

COZAAR PLUS


### Sugerir nombres.

In [7]:
import Levenshtein as lev

# # Sample database of words
# words_database = ["Eutirox","Eulitox", "Example", "Trombox", "Euphoria", "Euro"]

# # Function to suggest words
# def suggest_word(input_word, database, max_distance=4):
#     suggestions = []
#     for word in database:
#         if lev.distance(input_word.lower(), word.lower()) <= max_distance:
#             suggestions.append(word)
#     return suggestions

# Sample database of words
words_database = names

# Function to suggest words
def suggest_word(input_word, database, max_distance=3):
    suggestions = []
    for word in database:
        if lev.distance(input_word.lower(), word.lower()) <= max_distance:
            suggestions.append(word)
    return suggestions

# Testing the function
input_word = "eutirox"
suggestions = suggest_word(input_word, words_database)
print("Suggestions:", suggestions)

from fuzzywuzzy import process

def get_similar_words(query, word_list, limit=2):
    """
    Find similar words in a list to the given query word.
    
    :param query: The word to match.
    :param word_list: A list of words to search in.
    :param limit: Number of top matches to return. Defaults to 5.
    :return: List of tuples with matching word and similarity score.
    """
    # Use the process.extract function to find similar words
    similar_words = process.extract(query, word_list, limit=limit)
    return similar_words

# Example usage
word_list = names
query = "eutirox"
similar = get_similar_words(query, word_list)
print("Suggestions:", similar)

Suggestions: ['EUTIROX', 'GUTRON', 'ENTONOX', 'EULITOP', 'DUMIROX', 'ELMIRON', 'YATROX', 'REUTENOX', 'PENTHROX']
Suggestions: [('EUTIROX', 100), ('DUMIROX', 71)]
